# Library Usage in Seattle, 2005-2020

## Data Cleaning

The data that is the basis for this project is the [Checkouts by Title (Physical Items)](https://data.seattle.gov/Community/Checkouts-By-Title-Physical-Items-/5src-czff) dataset from [Seattle Open Data](https://data.seattle.gov/). It was downloaded on December 15, 2020.

This notebook is designed to load a downloaded CSV file, merge it with item-specific information, convert it to a time-series-ready DataFrame, and save that as a compressed pickle file.

*Note: This dataset is updated weekly; the more data, the longer the load times will be.*

In the future, it may be a good idea to look into adding [API](https://dev.socrata.com/foundry/data.seattle.gov/5src-czff) calls into the pipeline, so as to quickly and easily add on the additional weekly data.

*Note: Any cell that uses the built-in magic command* `%%time` *takes a significant (or at least not insignificant) time to run.*

## Table of contents

1. [Import required packages](#Import-required-packages)
2. [Load checkouts data](#Load-checkout-data)
3. [Convert date column to datetime](#Convert-date-column-to-datetime)
4. [Load data dictionary and merge onto checkouts dataset](#Load-data-dictionary-and-merge-onto-checkouts-dataset)
5. [Drop unnecessary columns](#Drop-unnecessary-columns)
6. [Transform values found during EDA](#Transform-values-found-during-EDA)
7. [Investigate categorical columns](#Investigate-categorical-columns)
8. [Transform values in two columns](#Transform-values-in-format_group-and-category_group)
9. [💾 Save](#💾-Save-in-11-parts)
10. [💾 Load](#💾-Load-in-11-parts)
11. [Checking for duplicates and assumption of data integrity](#Checking-for-duplicates-and-assumption-of-data-integrity)
12. [Dummy and count the necessary columns](#Dummy-and-count-the-necessary-columns)
13. [💾 Save/Load item counts DataFrame](#💾-Save/Load-item-counts-DataFrame)
14. [Next notebook: EDA](#Next-notebook:-EDA)

### Import required packages

In [1]:
# standard dataframe packages
import pandas as pd
import numpy as np

# custom functions
from functions.data_cleaning import *

# reload functions/libraries when edited
%load_ext autoreload
%autoreload 2

### Load checkouts data

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Since the data set is so large, I'll specify only the columns that I want in the DataFrame. This will effectively drop the following columns:
- `ID`
- `CheckoutYear`
- `CallNumber`
- `BibNumber`
- `ItemBarcode`
- `ItemType`
    
I want to note that the `ItemType` and `Collection` columns are very similar, but the code in the `Collection` column contains more information within the `category_group` column that I add onto the DataFrame using the `data_dictionary.csv` file ([see below](#Load-other-info-from-data-dictionary-and-merge-onto-checkouts-dataset)). More specifically, the `ItemType` code yields mostly "Miscellaneous" results, whereas the `Collection` code yields differentiates between "Fiction" and "Nonfiction", among others. This could be useful information later on, so I found it best to drop the `ItemType` column.

Note: In the future, I may consider using this [dataset](https://data.seattle.gov/Community/Library-Collection-Inventory/6vkj-f5xf) to add on branch information (i.e. which branch an item was checked out from), although this data is rather limited (due to privacy concerns) and incomplete (only appears to be collected beginning in 2017). In order to do that I would need to use the `BibNumber` column.

#### ⏰ Cell below takes ~15 minutes to run. ⏰

In [2]:
%%time

# columns to load
usecols = ['Collection', 'ItemTitle', 'Subjects', 'CheckoutDateTime']

# load data
df = pd.read_csv('data/Checkouts_By_Title__Physical_Items_.csv', usecols=usecols)

# rename columns to my preferred format
df.columns = ['collection', 'title', 'subjects', 'date']

CPU times: user 4min 26s, sys: 5min 21s, total: 9min 47s
Wall time: 14min 44s


*Note: While it only took about 25 seconds to load 10 million rows, it takes about 25 minutes to load 106.5 million rows with the same number (5) of columns. In my latest update, I brought it down to 4 columns, which decreased load time to 15.5 minutes.*

In [3]:
# check shape
df.shape

(106534901, 4)

In [4]:
# take a look
df.head()

,collection,title,subjects,date
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",02/13/2008 07:38:00 PM
1,nanf,best baby shower book a complete guide for par...,Showers Parties,07/23/2008 02:53:00 PM
2,nyfic,Uglies,"Fantasy, Teenage girls Fiction, Beauty Persona...",12/23/2009 04:20:00 PM
3,napar,doula guide to birth secrets every pregnant wo...,"Doulas, Childbirth",11/16/2010 12:04:00 PM
4,canf,Salmon a cookbook,Cookery Salmon,04/26/2009 01:29:00 PM


#### ⏰ Cell below takes ~4 minutes to run. ⏰

In [5]:
%%time

# check for nan values
df.isna().sum()

CPU times: user 44 s, sys: 1min 29s, total: 2min 13s
Wall time: 3min 47s


collection          0
title          900912
subjects      1649522
date                0
dtype: int64

*NOTE: Even checking for NaN values takes a significant amount of time with this many rows!*

The most important columns (`collection` and `date`) have no NaN values.

Also, [below](#Dummy-and-count-the-necessary-columns) I will account for these missing values by turning each column into a "missing-or-not" dummy column, in case there is any analysis to be drawn from that information, especially in relation to time.

In [6]:
# check datatypes
df.dtypes

collection    object
title         object
subjects      object
date          object
dtype: object

### Convert `date` column to datetime

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [7]:
# look at an example before conversion
df.loc[0, 'date']

'02/13/2008 07:38:00 PM'

In [8]:
# specify the format
dt_format = '%m/%d/%Y %I:%M:%S %p'

#### ⏰ Cell below takes ~7 minutes to run. ⏰

In [9]:
%%time

# convert to datetime, dropping the hour-minute-second stamp using the `dt.date` attribute
df['date'] = pd.to_datetime(df.date, format=dt_format).dt.date

# confirm it worked
df.loc[0, 'date']

CPU times: user 6min 6s, sys: 12.9 s, total: 6min 19s
Wall time: 6min 20s


datetime.date(2008, 2, 13)

### Load data dictionary and merge onto checkouts dataset

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [10]:
# load data
dd = pd.read_csv('data/data_dictionary.csv')

# rename columns to my preferred format
dd.columns = ['code', 'description', 'code_type', 'format_group', 'format_subgroup', 
              'category_group', 'category_subgroup', 'age_group']

# take a look
dd.head()

,code,description,code_type,format_group,format_subgroup,category_group,category_subgroup,age_group
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,NaN,Adult
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,NaN,Adult
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,NaN,Adult
3,nga,Northgate Branch,Location,NaN,NaN,NaN,NaN,NaN
4,hip,High Point Branch,Location,NaN,NaN,NaN,NaN,NaN


In [11]:
# check shape
dd.shape

(580, 8)

In [12]:
# check datatypes
dd.dtypes

code                 object
description          object
code_type            object
format_group         object
format_subgroup      object
category_group       object
category_subgroup    object
age_group            object
dtype: object

Since I will only be using information from codes whose type is "ItemCollection", I'll subset the data dictionary down to just those rows.

In [13]:
# subset to only collection codes
dd = dd[dd.code_type == 'ItemCollection']

In [14]:
# check for nan values
dd.isna().sum()

code                   0
description            0
code_type              0
format_group           0
format_subgroup       28
category_group         2
category_subgroup    391
age_group              0
dtype: int64

Again, with the size of the eventual DataFrame in mind, I want to drop any unnecessary columns before merging, so I'll drop the following columns:
- `description`, since that is superfluous information for this project
- `code_type`, since that is superfluous information
- `category_subgroup`, since that is mostly NaN values

In [15]:
# drop columns
dd.drop(columns=['description', 'code_type', 'category_subgroup'], inplace=True)

In [16]:
# list of columns to convert
to_convert = ['format_group', 'format_subgroup', 'category_group', 'age_group']

# convert to category datatype
dd[to_convert] = dd[to_convert].apply(pd.Categorical)

In [17]:
# confirm new datatypes
dd.dtypes

code                 object
format_group       category
format_subgroup    category
category_group     category
age_group          category
dtype: object

#### ⏰ Cell below takes ~4.5 minutes to run. ⏰

In [18]:
%%time

# merge checkouts dataframe with info from data dictionary
df_merged = df.merge(dd, left_on='collection', right_on='code')

# take a look
df_merged.head()

CPU times: user 1min, sys: 1min 53s, total: 2min 54s
Wall time: 4min 16s


,collection,title,subjects,date,code,format_group,format_subgroup,category_group,age_group
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",2008-02-13,nadvd,Media,Video Disc,Fiction,Adult
1,nadvd,Marley me,"Comedy films, Married people Drama, Philadelph...",2009-07-03,nadvd,Media,Video Disc,Fiction,Adult
2,nadvd,Six feet under The complete fourth season,"Video recordings for the hearing impaired, Pro...",2008-10-26,nadvd,Media,Video Disc,Fiction,Adult
3,nadvd,Doctor Who The next doctor,"London England Drama, Doctor Who Fictitious ch...",2010-11-10,nadvd,Media,Video Disc,Fiction,Adult
4,nadvd,School ties,"Antisemitism Drama, Video recordings for the h...",2008-12-28,nadvd,Media,Video Disc,Fiction,Adult


### Drop unnecessary columns

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

I can now drop the `collection` and `code` columns, since those are no longer necessary.

*NOTE: Using the Pandas method `.drop()` was taking well over an hour, so I'm going to try to subset it below, to see if that works any faster.*

#### ⏰ Cell below takes ~45 minutes to run. ⏰

In [19]:
%%time

# drop columns
df_merged.drop(columns=['collection', 'code'], inplace=True)

CPU times: user 3min 1s, sys: 19min 37s, total: 22min 38s
Wall time: 43min 57s


NOTE: I may be able to drop even more columns (thinking especially of `title` and `subjects`), since I'll mostly be looking at sheer numbers of items checked out each day. I'll keep them in for now in case they end up being useful for EDA.

### Transform values found during EDA

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

During EDA, I found the following values marked as `Print` in the `format_group` column and `Book` in the `format_subgroup` column, even though I believe they should be marked as `Equipment` and `Kit`:

- `SPL HotSpot connecting Seattle`
- `FlexTech Laptops`
- `In Building Device Checkout`

I will transform those values here.

In [20]:
# items to transform
to_transform = ['SPL HotSpot connecting Seattle', 'FlexTech Laptops',
                'In Building Device Checkout']

# custom function to transform format_group column
df_merged['format_group'] = transform_category(
    df_merged, 'title', 'format_group', to_transform, 'Equipment'
)

In [21]:
# custom function to transform format_subgroup column
df_merged['format_subgroup'] = transform_category(
    df_merged, 'title', 'format_subgroup', to_transform, 'Kit'
)

### Investigate categorical columns

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

As shown above, the data has no NaN values for the all important `date` column (the count of which will be the target for my eventual time series models), which is great news.

In terms of the categorical columns, I'll keep NaN values as they are, which means they simply won't factor into the category counts, but the items *will* still be counted in terms of how many items were checked out in a day.

In [22]:
# format_group breakdown
df_merged.format_group.value_counts()

Print         59615970
Media         46618209
Other           200452
Equipment        69211
Electronic           1
Name: format_group, dtype: int64

Since `Electronic` has so few observations, I'll lump it in with `Other` in the next section.

In [23]:
# format_subgroup breakdown
df_merged.format_subgroup.value_counts()

Book              59417481
Video Disc        30287406
Audio Disc        11238813
Audiobook Disc     2695078
Video Tape         1474457
Kit                 695201
Audiobook Tape      240328
Music Score         130486
Audio Tape           45946
Folder               23900
Data Disc             9886
Periodical             623
Document               471
Art                    129
Film                    81
Name: format_subgroup, dtype: int64

I think I can keep all these as is. They all seem to contain a fair amount of information and should be interesting as part of a dashboard or in EDA in general.

In [24]:
# category_group breakdown
df_merged.category_group.value_counts()

Fiction              65292861
Nonfiction           37539002
Miscellaneous         1734282
Language              1679452
Interlibrary Loan      192959
Reference               57420
On Order                 7422
Temporary                  40
WTBBL                      26
Periodical                  0
Name: category_group, dtype: int64

Some of these--`Miscellaneous`, `On Order`, `Temporary`--can be simplified into an `Other` category, since their current category doesn't provide any valuable information in terms of what the actual item is. I was originally considering including `Interlibrary Loan` in the `Other` category as well, but it may be interesting to see the numbers behind activity between the branches, so I'll leave it in for now.

Based on some research, `WTBBL` stands for "Washington Talking Book Library" and includes materials for folks with visual impairments. I was interested in looking into this, but since the numbers are so low, I think I'll also group that into the `Other` category, as I assume this `category_group` value deal with *equipment* that can be rented.

I will also convert `Periodical` to other; even though the count for it is 0, I assume this may relate to to items that have a `format_subgroup` value but no `category_group` value.

In [25]:
# category_group breakdown
df_merged.age_group.value_counts()

Adult       71587854
Juvenile    31006173
Teen         3909816
Name: age_group, dtype: int64

Everything looks good here!

### Transform values in `format_group` and `category_group`

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Here I'll consolidate a few values.

In [26]:
# values to convert
to_transform = ['Electronic']

# custom function to transform
df_merged['format_group'] = transform_category(
    df_merged, 'format_group', 'format_group', to_transform, 'Other'
)

# confirm
df_merged.format_group.unique()

[Media, Print, Equipment, Other]
Categories (4, object): [Media, Print, Equipment, Other]

In [27]:
# values to convert
to_transform = ['Miscellaneous', 'On Order', 'Temporary', 'WTBBL', 'Periodical']

# custom function to transform
df_merged['category_group'] = transform_category(
    df_merged, 'category_group', 'category_group', to_transform, 'Other'
)

# confirm
df_merged.category_group.unique()

[Fiction, Nonfiction, Language, Other, Interlibrary Loan, Reference, NaN]
Categories (6, object): [Fiction, Nonfiction, Language, Other, Interlibrary Loan, Reference]

### 💾 Save in 11 parts

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Due to some several kernel and zsh shell crashes, I'm going to try to save the DataFrame in batches of 10 million rows.

*NOTE: Save time for 10 million rows takes about 5 minutes and the file size is 290MB. Increasing to 20 million rows seemed to increase save time considerably, and so was interrupted before completing.*

#### ⏰ Cell below will most likely take ~10 hours to run. ⏰

In [28]:
%%time

# loop through index and multiples of 10 million
for ind, i in enumerate(range(0, 110000000, 10000000), 1):
    
    # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
    df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
    # print status/time
    status_update(f'File {ind} out of 11 saved successfully')

Current time = 01:18:45
-----------------------
File 1 out of 11 saved successfully

Current time = 01:24:19
-----------------------
File 2 out of 11 saved successfully

Current time = 01:50:08
-----------------------
File 3 out of 11 saved successfully

Current time = 02:00:07
-----------------------
File 4 out of 11 saved successfully

Current time = 03:20:17
-----------------------
File 5 out of 11 saved successfully

Current time = 03:48:17
-----------------------
File 6 out of 11 saved successfully

Current time = 04:10:44
-----------------------
File 7 out of 11 saved successfully

Current time = 05:30:07
-----------------------
File 8 out of 11 saved successfully

Current time = 06:43:37
-----------------------
File 9 out of 11 saved successfully

Current time = 08:20:01
-----------------------
File 10 out of 11 saved successfully

Current time = 09:07:15
-----------------------
File 11 out of 11 saved successfully

CPU times: user 46min 42s, sys: 2h 55min 10s, total: 3h 41min 5

If anything gets stuck or your computer crashes, uncomment the code below, replace `n` with the part to start the new loop with, and run the cell.

In [29]:
# %%time

# n = ____

# # loop through index and multiples of 10 million
# for ind, i in enumerate(range((n-1)*10000000, 110000000, 10000000), n):
    
#     # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
#     df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
#     # print status/time
#     status_update(f'File {ind} out of 11 saved successfully')

### 💾 Load in 11 parts

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

#### ⏰ Cell below takes ~25 minutes to run. ⏰

**Update**: I've turned the above loop into a function, which can be run below, after uncommenting the code.

In [30]:
%%time

# uncomment to load complete item checkouts data
df = load_multi_df(
    'data/', 'seattle_lib_', 'pkl', 11, compression='gzip', verbose=2
)

Current time = 09:07:17
-----------------------
Begin load...

Current time = 09:07:37
-----------------------
File 1 loaded successfully.

Current time = 09:07:48
-----------------------
File 2 loaded successfully.

Current time = 09:07:49
-----------------------
Concatenation successful. DataFrame consists of files 1-2.

Current time = 09:08:05
-----------------------
File 3 loaded successfully.

Current time = 09:08:07
-----------------------
Concatenation successful. DataFrame consists of files 1-3.

Current time = 09:08:19
-----------------------
File 4 loaded successfully.

Current time = 09:08:24
-----------------------
Concatenation successful. DataFrame consists of files 1-4.

Current time = 09:08:41
-----------------------
File 5 loaded successfully.

Current time = 09:08:55
-----------------------
Concatenation successful. DataFrame consists of files 1-5.

Current time = 09:09:12
-----------------------
File 6 loaded successfully.

Current time = 09:10:07
-------------------

In [31]:
# check shape
df.shape

(106503843, 7)

### Checking for duplicates and assumption of data integrity

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

The issue of checking for duplicates with this dataset is that duplicates are acceptable! It is very likely that the same item is checked out from either the same or different branches on a single day. Multiple copies of a book, for example, can be stored in one branch or across several branches.

NOTE: More investigation on the uniqueness of an item's call number could potentially solve this and allow me to check for actual duplicate rows. For the time being, I will assume the data is almost entirely, if not entirely, accurate.

Checking for duplicates can be done by uncommenting and running the code below, although it may take quite awhile to run.

In [32]:
# %%time

# df[df.duplicated(keep=False)]

### Dummy and count the necessary columns

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [33]:
# columns to dummy
dummy_cols = ['format_group', 'format_subgroup', 'category_group', 'age_group']

# dummy the columns
dummy_df = pd.get_dummies(df[dummy_cols], prefix=dummy_cols)

# take a look
dummy_df.head()

,format_group_Equipment,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,format_subgroup_Audiobook Disc,format_subgroup_Audiobook Tape,format_subgroup_Book,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


I'm going to also create columns containing information on whether or not a value for the `title` and/or `subjects` column is missing. My hope is that, when analyzing this information in terms of time, I can get a sense of whether missing item information is a persistent issue, or if it has been corrected.

#### ⏰ Cell below takes ~2 minutes to run. ⏰

In [39]:
%%time

# `1` if title is missing, `0` if not
df['missing_title'] = np.where(df.title.isna(), 1, 0)

# `1` if subjects is missing, `0` if not
df['missing_subjects'] = np.where(df.subjects.isna(), 1, 0)

CPU times: user 18 s, sys: 54.9 s, total: 1min 12s
Wall time: 1min 56s


In [40]:
%%time

# combine with date column
df_counts = pd.concat([df[['date', 'missing_title', 'missing_subjects']], dummy_df], axis=1)

# take a look
df_counts.head()

CPU times: user 7.14 s, sys: 18.2 s, total: 25.3 s
Wall time: 34.1 s


,date,missing_title,missing_subjects,format_group_Equipment,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
0,2008-02-13,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,2009-07-03,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,2008-10-26,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,2010-11-10,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,2008-12-28,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [41]:
# check shape
df_counts.shape

(106503843, 31)

#### ⏰ Cell below takes ~2 minutes to run. ⏰

In [42]:
%%time

# group by date and get category total for each column
df_counts = df_counts.groupby('date').agg('sum')

# take a look
df_counts.head()

CPU times: user 42.2 s, sys: 52.8 s, total: 1min 34s
Wall time: 1min 53s


,missing_title,missing_subjects,format_group_Equipment,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,format_subgroup_Audiobook Disc,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
date,,,,,,,,,,,,,,,,,,,,,
2005-04-13,212,664,1.0,6397.0,32.0,10041.0,0.0,1874.0,63.0,217.0,...,1878.0,8189.0,32.0,370.0,6719.0,1143.0,18.0,11257.0,4613.0,601.0
2005-04-14,123,541,1.0,4015.0,75.0,6267.0,0.0,1245.0,31.0,164.0,...,1115.0,5276.0,73.0,272.0,4104.0,621.0,12.0,6726.0,3381.0,251.0
2005-04-15,179,508,0.0,5351.0,51.0,7494.0,0.0,1462.0,54.0,187.0,...,1721.0,6357.0,50.0,302.0,5166.0,1014.0,7.0,8795.0,3747.0,354.0
2005-04-16,7,56,0.0,552.0,0.0,806.0,0.0,175.0,8.0,31.0,...,163.0,567.0,0.0,29.0,666.0,95.0,1.0,950.0,367.0,41.0
2005-04-17,80,232,0.0,1555.0,8.0,2992.0,0.0,499.0,10.0,47.0,...,480.0,2017.0,8.0,177.0,2145.0,203.0,5.0,3035.0,1349.0,171.0


In [43]:
# combine with total checkouts per day
df_counts = pd.concat([df.groupby('date').size(), df_counts], axis=1)

# rename target column
df_counts.columns = ['total_checkouts'] + list(df_counts.columns[1:])

# take a look
df_counts.head()

,total_checkouts,missing_title,missing_subjects,format_group_Equipment,format_group_Media,format_group_Other,format_group_Print,format_subgroup_Art,format_subgroup_Audio Disc,format_subgroup_Audio Tape,...,format_subgroup_Video Tape,category_group_Fiction,category_group_Interlibrary Loan,category_group_Language,category_group_Nonfiction,category_group_Other,category_group_Reference,age_group_Adult,age_group_Juvenile,age_group_Teen
date,,,,,,,,,,,,,,,,,,,,,
2005-04-13,16471,212,664,1.0,6397.0,32.0,10041.0,0.0,1874.0,63.0,...,1878.0,8189.0,32.0,370.0,6719.0,1143.0,18.0,11257.0,4613.0,601.0
2005-04-14,10358,123,541,1.0,4015.0,75.0,6267.0,0.0,1245.0,31.0,...,1115.0,5276.0,73.0,272.0,4104.0,621.0,12.0,6726.0,3381.0,251.0
2005-04-15,12896,179,508,0.0,5351.0,51.0,7494.0,0.0,1462.0,54.0,...,1721.0,6357.0,50.0,302.0,5166.0,1014.0,7.0,8795.0,3747.0,354.0
2005-04-16,1358,7,56,0.0,552.0,0.0,806.0,0.0,175.0,8.0,...,163.0,567.0,0.0,29.0,666.0,95.0,1.0,950.0,367.0,41.0
2005-04-17,4555,80,232,0.0,1555.0,8.0,2992.0,0.0,499.0,10.0,...,480.0,2017.0,8.0,177.0,2145.0,203.0,5.0,3035.0,1349.0,171.0


### 💾 Save/Load item counts DataFrame 

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [44]:
# uncomment to save
df_counts.to_pickle(f'data/seattle_lib_counts.pkl', compression='gzip')

# # uncomment to load
# df_counts = pd.read_pickle('data/seattle_lib_counts.pkl', compression='gzip')

Wow, how incredible to have been able to distill that gigantic dataset into one that takes less than a second to save/load and is all of 257 KB.

# Next notebook: [EDA](02_eda.ipynb)

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

- The next notebook includes exploring and analyzing both the larger checkouts dataset and the item counts dataset.